In [144]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup, Comment
import re
import pandas as pd
import os
import numpy as np

from urllib.parse import urlparse, parse_qs
import os.path

import random

import time

import ast

import nltk
from nltk import sentiment, word_tokenize, pos_tag, ne_chunk
from nltk.corpus import opinion_lexicon
from nltk.tokenize import treebank

In [123]:
%matplotlib inline

In [2]:
# nltk.download('maxent_ne_chunker')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')
# nltk.download('words')
# nltk.download('opinion_lexicon')

In [43]:
def load_facebook():
    url = "https://facebook.com"
    driver = webdriver.Firefox()
    driver.get(url)
    driver.find_element_by_id('email').send_keys('n00bdan13@gmail.com')
    driver.find_element_by_id('pass').send_keys('daniel124')
    driver.find_element_by_id('pass').send_keys(Keys.ENTER)
    
    try:
        elem = WebDriverWait(driver, 2).until(EC.title_contains("Facebook"))
    except TimeoutException:
        print("Too much time")
    
    return driver

def get_nltk_sentiment(sentence, method):
        
    if (method == 'vader'):
        sa = sentiment.vader.SentimentIntensityAnalyzer()
        output = sa.polarity_scores(str(sentence))

        return output['compound']
    
    elif (method == 'liu'):
        
        wordType = ''
        
        if "PERSON" in str(ne_chunk(pos_tag(word_tokenize(sentence)))):
            wordType = 'tag'
        
        tokenizer = treebank.TreebankWordTokenizer()
        pos_words = 0
        neg_words = 0
        tokenized_sent = [word.lower() for word in tokenizer.tokenize(sentence)]
        
        for word in tokenized_sent:
            if word in opinion_lexicon.positive():
                pos_words += 1
            elif word in opinion_lexicon.negative():
                neg_words += 1
                
        if pos_words > neg_words:
            return 'Positive'
        elif pos_words < neg_words:
            return 'Negative'
        elif pos_words == neg_words:
            if wordType == 'tag':
                return 'Positive'
            else:
                return 'Neutral'

def get_comments(url, driver):
    
    pagePath = urlparse(url).path
    pName = pagePath.split("/")[1]
    
    driver.get(url)
    
    try:
        elem = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.CLASS_NAME, "fbPhotoSnowliftContainer")))
    except TimeoutException:
        print("Too much time")
    
    while True:
        time.sleep(2)
        
        try:
            driver.find_elements_by_xpath("(//a[contains(text(),'View more comments')])")[3].click()
        except:
            break
            
    html = driver.page_source

    soup = BeautifulSoup(html, "lxml")

    comments = soup.find_all("div", class_="UFICommentContent")

    df = pd.DataFrame(columns=['Comment', 'Profile URL'])

    for comment in comments:
        body = comment.find("span", class_="UFICommentBody").getText()
        currURL = comment.find('a', href=True)['href']
        
        if (currURL.find(pName) == -1):   
            df = df.append({'Comment': body, 'Profile URL': currURL}, ignore_index=True)
            

    return df

def get_positive(url, driver):
    df = get_comments(url, driver)
    df['Sentiment'] = df['Comment'].apply(get_nltk_sentiment, method="liu")
    newDf = df[df['Sentiment'] == 'Positive'].drop_duplicates('Profile URL', keep='first')
    
    newDf = newDf.reset_index(drop=True)
    return newDf

def get_id(text):
    parsed = urlparse(text)
    parsed_dict = parse_qs(parsed.query)
    userId = parsed_dict['id']

    return userId[0]

def get_userid(url, driver):
    ret_id = ''
    
    if ("profile.php?id=" in url):
        ret_id = get_id(url)
    else:
        driver.get(url)
        time.sleep(4)

        try:
            elem = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.CLASS_NAME, 'hidden_elem')))
        except TimeoutException:
            print("Too much time")

        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")

        userId = ''

        for aVals in soup.find_all('a'):
            try:
                text = aVals['data-hovercard']
                ret_id = get_id(text)
                
                if ("/ajax/hovercard/user.php?id={}".format(ret_id) in text):
                    break
            except:
                pass
            
    return ret_id

In [44]:
def scroll_till_bottom(driver):
    count = 0
    
    SCROLL_PAUSE_TIME = 0.7

    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # Scroll down to bottom
        time.sleep(SCROLL_PAUSE_TIME) # Wait to load page

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            count = count + 1
            
            if (count > 5):
                print('Probably reached end of result trying again')
                break
        else:
            print("Loading new results")
            count = 0
            
        last_height = new_height #while Loading more results... exists
        
def scrape_likes(user,driver):
    url = 'https://www.facebook.com/search/{}/pages-liked/'.format(user)
    driver.get(url)
    
    linkList = []
    
    try:
        elem = WebDriverWait(driver, 2).until(EC.title_contains("Pages liked by"))
    except TimeoutException:
        print("Too much time")
    
    time.sleep(3)
    
    scroll_till_bottom(driver)

    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")

    if "We couldn\'t find anything for" in str(soup):
        print("No likes found for {}".format(user))
    else:
        res = soup.find("div", {"id": "browse_result_area"})

        links = res.find_all('a')
        
        scroll_till_bottom(driver)
        
        for link in links:
            if ('facebook.com' in link['href']):
                if (link['href'] not in linkList):
                    linkList.append(link['href'])
                
    return linkList

In [60]:
def perform_scraping(df):
    df['ProfileID'] = ""
    df['PagesLiked'] = ""

    for idx, row in df.iterrows():
        print("Getting UserID for {}".format(row['Profile URL']))
        profileId = get_userid(row['Profile URL'], driver)
        df.at[idx, 'ProfileID'] = profileId
        print("The profile id is: {}".format(profileId))

        sleepsecs = random.randint(2,6)
        print("Sleeping for {} seconds".format(sleepsecs))
        time.sleep(sleepsecs)

        print("Getting pages liked by {}".format(profileId))
        pagesLiked = scrape_likes(profileId, driver)
        df.at[idx, 'PagesLiked'] = pagesLiked
        print("The pages liked are {}".format(pagesLiked))

        sleepsecs = random.randint(2,6)
        print("Sleeping for {} seconds".format(sleepsecs))
        time.sleep(sleepsecs)

        print('\n')

        df.to_csv('currentlogs.csv', index=False)

Getting UserID for https://www.facebook.com/valerie.glasgowlucas?fref=ufi&rc=p
The profile id is: 100000670838598
Sleeping for 4 seconds
Getting pages liked by 100000670838598
Probably reached end of result trying again
Probably reached end of result trying again
The pages liked are ['https://www.facebook.com/foodbyshared/?ref=br_rs', 'https://www.facebook.com/search/109870912368806/places-in/2233/places/intersect/', 'https://www.facebook.com/ThinkingHumanity/?ref=br_rs', 'https://www.facebook.com/PatientsRising/?ref=br_rs', 'https://www.facebook.com/search/110184922344060/places-in/2603/places/intersect/', 'https://www.facebook.com/promixgardening/?ref=br_rs', 'https://www.facebook.com/search/110925022261358/places-in/156555961070638/places/intersect/', 'https://www.facebook.com/diplyhumor/?ref=br_rs', 'https://www.facebook.com/TrilogyWickenburgRanch/?ref=br_rs', 'https://www.facebook.com/search/104047142964824/places-in/198327773511962/places/intersect/', 'https://www.facebook.com/Fo



Getting UserID for https://www.facebook.com/betty.rogers.5895?fref=ufi&rc=p
The profile id is: 100000644108284
Sleeping for 6 seconds
Getting pages liked by 100000644108284
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loadi



Getting UserID for https://www.facebook.com/dulu.boyd?fref=ufi&rc=p
The profile id is: 1235595588
Sleeping for 4 seconds
Getting pages liked by 1235595588
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Probably reached end of result trying again
Probably reached end of result trying again
The pages liked are ['https://www.f



Getting UserID for https://www.facebook.com/profile.php?id=100009751844050&fref=ufi&rc=p
The profile id is: 100009751844050
Sleeping for 4 seconds
Getting pages liked by 100009751844050
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Loading new results
Probably rea



Getting UserID for https://www.facebook.com/mrs.tina.l.mixon?fref=ufi&rc=p
The profile id is: 1075832673
Sleeping for 6 seconds
Getting pages liked by 1075832673


KeyboardInterrupt: 

In [254]:
def get_list(liststr):
    try:
        pageList = ast.literal_eval(liststr)
        
        pNames = []
        
        for page in pageList:
            pName = page.split("/")[3]
            
            if (pName == 'pages'):
                pName = page.split("/")[4]
            elif (pName == 'search'):
                pName = page.split("/")[5]
                
            pNames.append(pName)
            
        
        return pNames
    except:
        return []

In [255]:
def get_pagesusers(df):
    allPages = []

    for idx, row in df.iterrows():
        for page in row['PageName']:
            allPages.append(page)

    uniquePages = list(set(allPages))

    userIds = []

    for idx, row in df.iterrows():
        try:
            row['ProfileID'] = int(row['ProfileID'])
        except:
            row['ProfileID'] = row['ProfileID']

        userIds.append(row['ProfileID'])

    userIds.insert(0, "Page Name")
    
    return allPages, userIds

In [ ]:
driver = load_facebook()

In [ ]:
commentsDf = get_positive('https://www.facebook.com/promisedlanddairy/photos/a.328814060485906.84135.179461015421212/1896136953753601/?type=3&theater', driver)

In [ ]:
perform_scraping(commentsDf)

In [ ]:
userDf = pd.read_csv('currentlogs.csv')

if ('Unnamed: 0' in userDf):
    userDf = userDf.drop('Unnamed: 0', axis=1)

userDf['PagesLiked'] =  userDf['PagesLiked'].astype(str)
userDf['PageName'] = userDf['PagesLiked'].apply(get_list)

In [ ]:
pages, ids = get_pagesusers(userDf)

In [252]:
finalDf = pd.DataFrame(columns=ids)

finalDf['Page Name'] = uniquePages
finalDf = finalDf.set_index('Page Name')

for profileID in finalDf.columns:
    pagesLiked = df[df['ProfileID'] == profileID]['PageName']
    finalDf[profileID] = False
    
    
    for pages in pagesLiked:
        for page in set(pages):
            print(page)
            print(profileID)
            finalDf.at[profileID, page] = True

democraticunderground
100000670838598
julianabophotography
100000670838598
foodbyshared
100000670838598
FoodsAround
100000670838598
nutrish
100000670838598
JeffMerkleyOregon
100000670838598
MonicaLaliberte5OYS
100000670838598
nOHMatters
100000670838598
ThinkingHumanity
100000670838598
diplyhumor
100000670838598
PatientsRising
100000670838598
LiberalMountain
100000670838598
places-in
100000670838598
TrilogyWickenburgRanch
100000670838598
YourDailyDish
100000670838598
Goff-Roof-Systems-244597728887239
100000670838598
Royal-Canadian-Legion-Branch-98-1485632075083765
100000670838598
HerculesRadioCo
100000670838598
promixgardening
100000670838598
peticaljubimci
100017761387060
JunesJourney
100017761387060
i.lovebirds.andflowers
100017761387060
places-in
100017761387060


AttributeError: 'BlockManager' object has no attribute 'T'

In [253]:
#manage places-in